In [49]:
#import libraries.
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
# import keras
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
#import sounddevice as sd
# import tensorflow as tf
# tf.enable_eager_execution()
from pystoi.stoi import stoi
import h5py
######################
#import libraries.
import matplotlib.pyplot as plt
from tabulate import tabulate
import time
import os
import librosa
from librosa.core import stft, istft
####import sounddevice as sd
import time
print('imported')
# #######################
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
input_tr_name = 'ftr_refrmd_10h'
input_val_name = 'ftr_refrmd_test'
target_tr_name = 'single_dataset_log_16'
target_val_name = 'clean_data'
tfrecord_folder_parent = 'tfrecord_files'
tfrecord_folder = 'tfrecord_files_10h'
tfrecord_val_folder = 'test'
input_tr_path = os.path.normpath(os.path.join(Data_path,input_tr_name))
input_val_path = os.path.normpath(os.path.join(Data_path,input_val_name))
target_tr_path = os.path.normpath(os.path.join(Data_path,target_tr_name))
target_val_path = os.path.normpath(os.path.join(Data_path,target_val_name))

file = h5py.File(target_val_path+'.hdf5','r')
val_y = file[target_val_name][0:]
file.close()
file = h5py.File(input_val_path+'.hdf5','r')
val_x = file[input_val_name][0:]
file.close()

hh = h5py.File(target_tr_path+'.hdf5', 'r')
d=hh[target_tr_name]
len_data=d.shape
hh.close()

file = h5py.File(target_val_path+'.hdf5','r')
val_y = file[target_val_name]
val_len=val_y.shape
file.close()

w=3
# print(len_data)
#######################
#define reconstruct function to reconstruct sound from framed signal.
def reconstruct(wave,angle):
    recon = np.sqrt(np.power(10, wave))
    recon1 = recon*np.cos(angle)+recon*np.sin(angle)*1j
    recon = librosa.core.istft((recon1.T), hop_length=200, win_length=500, window='hann')
    return recon
#######################
I=0
global batch_size
batch_size=128
# epochs_num=50
global datalen
datalen=len_data[0]

file.close()
h = [1024,512]
seed = 7
from tensorflow.keras.layers import Activation
np.random.seed(seed)
model = Sequential()
act=layers.LeakyReLU(alpha=0.1)
model.add(Dense(h[0], input_dim = w*len_data[1]))
# model.add(BatchNormalization())
model.add(act)
act=layers.LeakyReLU(alpha=0.1)
model.add(Dense(h[1]))
model.add(act)
act=layers.LeakyReLU(alpha=0.1)
model.add(Dense(len_data[1]))
#############################################
import os
from natsort import natsorted

def _parse_function(example_proto):
    features = {"X": tf.FixedLenFeature((3*257), tf.float32),
              "Y": tf.FixedLenFeature((257), tf.float32)}
    parsed_features = tf.parse_single_example(example_proto, features)
    print("i was here")
    return parsed_features["X"], parsed_features["Y"]

# orig_path = os.getcwd()
tfrecord_path = os.path.normpath(os.path.join(Data_path,tfrecord_folder_parent,tfrecord_folder))
sorted_names = natsorted(os.listdir(tfrecord_path))
trainfilenames = []
for i in sorted_names:
    trainfilenames.append(os.path.normpath(os.path.join(tfrecord_path,i)))
filenames = tf.placeholder(tf.string, shape=[None])
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(_parse_function)  # Parse the record into tensors.
dataset = dataset.repeat()  # Repeat the input indefinitely.
dataset = dataset.batch(128)
iterator = dataset.make_initializable_iterator()


tfrecord_path_val = os.path.normpath(os.path.join(Data_path,tfrecord_folder_parent,tfrecord_val_folder))
sorted_names_val = natsorted(os.listdir(tfrecord_path_val))
trainfilenames_val = []
for i in sorted_names_val:
    trainfilenames_val.append(os.path.normpath(os.path.join(tfrecord_path_val,i)))
filenames_val = tf.placeholder(tf.string, shape=[None])
dataset_val = tf.data.TFRecordDataset(filenames_val)
dataset_val = dataset_val.map(_parse_function)  # Parse the record into tensors.
dataset_val = dataset_val.repeat()  # Repeat the input indefinitely.
dataset_val = dataset_val.batch(128)
iterator_val = dataset_val.make_initializable_iterator()


# You can feed the initializer with the appropriate filenames for the current
# phase of execution, e.g. training vs. validation.
# next_elem = iterator_val.get_next()
# Initialize `iterator` with training data.
with tf.Session() as sess:
    sess.run(iterator.initializer, feed_dict={filenames: trainfilenames})
    sess.run(iterator_val.initializer, feed_dict={filenames_val: trainfilenames_val})
    print("initialized")
    model.compile(loss='mean_squared_error', optimizer='adam')
    epochs_num=50
    steps = len_data[0] // batch_size
    val_steps = val_len[0] // batch_size
    history = model.fit( iterator, steps_per_epoch=steps,epochs=epochs_num, verbose=1,validation_data=iterator_val,validation_steps=val_steps)
#     history = model.fit( iterator, steps_per_epoch=steps,epochs=epochs_num, verbose=1)
#     a=sess.run(next_elem[0])
#     print(a.shape)
#############################################
# I=0
# def da():
#     load_size = batch_size
#     data_len=datalen
#     global I
#     h5f1 = h5py.File('ftr_refrmd_10h.hdf5','r')
#     h5f2 = h5py.File('single_dataset_log_16.hdf5','r')
#     indx = np.arange(0, data_len, 1)
#     indx = shuffle(indx, random_state=1)
#     print(indx)
#     while True:             #this line is just because keras needs infinite generators
#         for I in indx: 
#             Data1 = h5f1['ftr_refrmd_10h'][I]
#             Data2 = h5f2['single_dataset_log_16'][I]
#             yield (np.reshape(Data1,(1,len(Data1))), np.reshape(Data2,(1,len(Data2))))
            
# ds = tf.data.Dataset.from_generator(
#     da, (tf.float32, tf.float32), ((1,257*w), (1,257)))

#############################################
# is_training=True
# h5f1 = h5py.File('ftr_refrmd_10h.hdf5','r')
# h5f2 = h5py.File('single_dataset_log_16.hdf5','r')
# def get_record(i):
#     # load the ith record using standard python, return numpy arrays
#     Data1 = h5f1['ftr_refrmd_10h'][i]
#     Data2 = h5f2['single_dataset_log_16'][i]
#     return (Data1, Data2)

# def tf_map_fn(index):
#     features, labels = tf.py_func(
#         get_record, (index,), (tf.float32, tf.float32), stateful=False)
#     features.set_shape((w*len_data[1],))
#     labels.set_shape((len_data[1]))
# #     do data augmentation here
#     return features, labels
# # global epochs_num
# epochs_num=30
# epoch_size = epochs_num
# dataset = tf.data.Dataset.from_tensor_slices(get_record)
# if is_training:
#     dataset = dataset.repeat().shuffle(epoch_size)
# dataset = dataset.map(tf_map_fn)
# dataset = dataset.batch(batch_size)
# # prefetch data to CPU while GPU processes previous batch
# dataset = dataset.prefetch(1)
##############################################
# h5f1val = h5py.File('ftr_refrmd_test.hdf5','r')
# h5f2val = h5py.File('clean_data.hdf5','r')
# def get_record(i):
#     # load the ith record using standard python, return numpy arrays
#     Data1 = h5f1val['ftr_refrmd_test'][i]
#     Data2 = h5f2val['clean_data'][i]
#     return (Data1, Data2)

# def tf_map_fn(index):
#     features, labels = tf.py_func(
#         get_record, (index,), (tf.float32, tf.float32), stateful=False)
#     features.set_shape((w*len_data[1],))
#     labels.set_shape((len_data[1]))
# #     do data augmentation here
#     return features, labels
# # global epochs_num
# epochs_num=30
# epoch_size = epochs_num
# dataset_val = tf.data.Dataset.from_tensor_slices((tf.range(epoch_size,)))
# if is_training:
#     dataset_val = dataset_val.repeat().shuffle(epoch_size)
# dataset_val = dataset_val.map(tf_map_fn)
# dataset_val = dataset_val.batch(batch_size)
# # prefetch data to CPU while GPU processes previous batch
# dataset_val = dataset_val.prefetch(1)
############################****************************#######################
# ds=tf.data.Dataset.from_generator(
#         da, (tf.float32, tf.float32))
# model.compile(loss='mean_squared_error', optimizer='adam')
# epochs_num=50
# steps = len_data[0] // batch_size
# val_steps = val_len[0] // batch_size
# # history = model.fit( dataset, steps_per_epoch=steps,epochs=epochs_num, verbose=1,validation_data=dataset_val,validation_steps=val_steps)
# history = model.fit( iterator, steps_per_epoch=steps,epochs=epochs_num, verbose=1)
%matplotlib inline
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()
model_json = model.to_json()
with open("model_10h_dataset.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model_10h_dataset.h5")
print("Saved model to disk")

imported
i was here
i was here
initialized
Epoch 1/50
  104/18706 [..............................] - ETA: 13:48 - loss: 2.0206

KeyboardInterrupt: 

In [39]:
os.path.join(tfrecord_path,sorted_names)

TypeError: join() argument must be str or bytes, not 'list'

In [38]:
tfrecord_path

'D:\\studies\\university\\thesis\\speech_separation_codes\\du16\\donesomestuff\\tfrecord_files\\tfrecord_files_10h'

In [27]:
a=tf.constant([1,2,3])
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    label_numpy = a.eval()

In [97]:
import h5py 
import tensorflow as tf
hh = h5py.File('ftr_refrmd_10h.hdf5', 'r')
d=hh['ftr_refrmd_10h'][0]
len_data=d.shape
hh.close()
len_data

(771,)

In [24]:
file.close()

In [92]:
dataset

<PrefetchDataset shapes: ((?, 257), (?,)), types: (tf.float32, tf.float32)>